In [36]:
import os
import sys
import pandas as pd
import pyarrow as pa
import pyarrow.csv as pc
import pyarrow.parquet as pq
from abc import ABC, abstractmethod
import logging
from tqdm import tqdm

In [37]:
PMTfied_dir = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied/"
truth_18_1 = PMTfied_dir + "Snowstorm/22018/truth_1.parquet"
pmtfied_18_1_1 = PMTfied_dir + "Snowstorm/22018/1/PMTfied_1.parquet"

truth_11_2 = PMTfied_dir + "Snowstorm/22011/truth_2.parquet"

truth_14_1 = PMTfied_dir + "Snowstorm/22014/truth_1.parquet"

In [38]:

source_root = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied/"
dest_root = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied_filtered/"
snowstorm_source_dir = source_root + "Snowstorm/"
snowstorm_dest_dir = dest_root + "Snowstorm/"

corsika_source_dir = source_root + "Corsika/"
corsika_dest_dir = dest_root + "Corsika/"
dir_99999 = snowstorm_dest_dir + "99999/"
dir_99999_98 = dir_99999 + "98/"
dir_99999_99 = dir_99999 + "99/"

dir_99999_Corsika = corsika_dest_dir+"9999999-9999999/"
dir_99999_Corsika_96 = dir_99999_Corsika + "96/"
dir_99999_Corsika_97 = dir_99999_Corsika + "97/"

In [39]:
pure_nu_specifier_dir = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/"
pure_nu_specifier_18_1 = pure_nu_specifier_dir + "2018/22018/clean_event_ids_0000000-0000999.csv"
pure_nu_specifier_11_1 = pure_nu_specifier_dir + "2011/22011/clean_event_ids_0000000-0000999.csv"


In [40]:
JanniFile_22010_1 = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22010/22010/clean_event_ids_0000000-0000999.csv"
JanniFile_22011_1 = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22011/22011/clean_event_ids_0000000-0000999.csv"
JanniFile_22010_1_reduction = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22010/22010/reduced/clean_event_ids_0000000-0000999.csv"
JanniFile_22011_1_reduction = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22011/22011/reduced/clean_event_ids_0000000-0000999.csv"

In [41]:
# find /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/ -type f -name "*.csv" -exec sed -i 's/\r$//' {} \;

In [42]:
'''
cat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22011/22011/clean_event_ids_0000000-0000999.csv | head -n 10
cat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22012/22012/clean_event_ids_0000000-0000999.csv | head -n 10
cat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22010/22010/reduced/clean_event_ids_0000000-0000999.csv | head -n 10
cat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22011/22011/reduced/clean_event_ids_0000000-0000999.csv | head -n 10
cat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22012/22012/reduced/clean_event_ids_0000000-0000999.csv | head -n 10
'''

'\ncat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22011/22011/clean_event_ids_0000000-0000999.csv | head -n 10\ncat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22012/22012/clean_event_ids_0000000-0000999.csv | head -n 10\ncat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22010/22010/reduced/clean_event_ids_0000000-0000999.csv | head -n 10\ncat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22011/22011/reduced/clean_event_ids_0000000-0000999.csv | head -n 10\ncat -A /lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22012/22012/reduced/clean_event_ids_0000000-0000999.csv | head -n 10\n'

In [43]:
# df_JanniFile_22010_1 = pd.read_csv(JanniFile_22010_1)
# df_JanniFile_22010_1.head()
# # 25sec

In [44]:
df_JanniFile_22011_1 = pd.read_csv(JanniFile_22011_1)
df_JanniFile_22011_1.head()

,Run ID,Subrun ID,Event ID,Subevent ID,Subevent Stream
0,2201100000,4294967295,10,0,NaN
1,2201100000,4294967295,23,0,NaN
2,2201100000,4294967295,29,0,NaN
3,2201100000,4294967295,40,0,NaN
4,2201100000,4294967295,45,0,NaN


In [45]:
try:
    df = pd.read_csv(JanniFile_22010_1_reduction, dtype={'RunID': str, 'EventID': str})  # Read as string to catch format issues
    print("\n### First 5 Rows of the File ###")
    print(df.head())

    print("\n### DataFrame Info ###")
    print(df.info())

    print("\n### Unique RunID Sample ###")
    print(df['RunID'].unique()[:10])  # Print first 10 unique RunIDs for checking

    print("\n### Summary Statistics ###")
    print(df.describe(include='all'))
    
    df_invalid = df[~df['RunID'].str.match(r'^\d+$', na=False)]
    print("\n### Invalid RunIDs (Non-Numeric) ###")
    print(df_invalid)
    
    print("\n### Missing RunIDs ###")
    missing_runids = set(range(2201000000, 2201001000)) - set(df['RunID'].astype(int).unique())
    print(sorted(missing_runids))

    print("\n### Top 10 Most Frequent RunIDs ###")
    print(df['RunID'].value_counts().head(10))

    df_trimmed = df[df['RunID'] != df['RunID'].str.strip()]
    print("\n### RunIDs with Extra Spaces ###")
    print(df_trimmed)


except Exception as e:
    print("Error reading the CSV file:", e)

# 18sec

Error reading the CSV file: [Errno 2] No such file or directory: '/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/clean_events_dict/22010/22010/reduced/clean_event_ids_0000000-0000999.csv'


In [46]:
df_JanniFile_22011_1_reduction = pd.read_csv(JanniFile_22011_1_reduction)
df_JanniFile_22011_1_reduction.head()

,RunID,EventID
0,2201100000,10
1,2201100000,23
2,2201100000,29
3,2201100000,40
4,2201100000,45


In [47]:
filtered_dir = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied_filtered/"
selected_events_dir = filtered_dir + "PureNeutrinos/"

In [48]:
def convertParquetToDF(file:str) -> pd.DataFrame:
    table = pq.read_table(file)
    df = table.to_pandas()
    return df

In [49]:
df_truth_18_1 = convertParquetToDF(truth_18_1)
df_truth_11_2 = convertParquetToDF(truth_11_2)
df_truth_14_1 = convertParquetToDF(truth_14_1)

In [50]:
df_truth_11_2.columns   

Index(['event_no', 'original_event_no', 'subdirectory_no', 'part_no',
       'shard_no', 'N_doms', 'offset', 'energy', 'azimuth', 'zenith', 'pid',
       'event_time', 'interaction_type', 'elasticity', 'RunID', 'SubrunID',
       'EventID', 'SubEventID', 'dbang_decay_length', 'track_length',
       'stopped_muon', 'energy_track', 'energy_cascade', 'inelasticity',
       'DeepCoreFilter_13', 'CascadeFilter_13', 'MuonFilter_13',
       'OnlineL2Filter_17', 'L3_oscNext_bool', 'L4_oscNext_bool',
       'L5_oscNext_bool', 'L6_oscNext_bool', 'L7_oscNext_bool',
       'Homogenized_QTot', 'MCLabelClassification', 'MCLabelCoincidentMuons',
       'MCLabelBgMuonMCPE', 'MCLabelBgMuonMCPECharge',
       'GNLabelTrackEnergyDeposited', 'GNLabelTrackEnergyOnEntrance',
       'GNLabelTrackEnergyOnEntrancePrimary',
       'GNLabelTrackEnergyDepositedPrimary', 'GNLabelEnergyPrimary',
       'GNLabelCascadeEnergyDepositedPrimary', 'GNLabelCascadeEnergyDeposited',
       'GNLabelEnergyDepositedTotal', 'GN

In [51]:
df_truth_11_2['event_no']

0         111000200400561
1         111000200400562
2         111000200400563
3         111000200400564
4         111000200400565
               ...       
402680    111000200803241
402681    111000200803242
402682    111000200803243
402683    111000200803244
402684    111000200803245
Name: event_no, Length: 402685, dtype: int64

In [52]:
df_truth_11_2['RunID'].sort_values().unique()

array([2201100014, 2201100015, 2201100016, 2201100017, 2201100018,
       2201100019, 2201100020, 2201100024, 2201100045, 2201100049,
       2201100050, 2201100054, 2201100055, 2201100056, 2201100057,
       2201100058, 2201100084, 2201100085, 2201100086, 2201100087,
       2201100088, 2201100089, 2201100090, 2201100094, 2201100095,
       2201100096, 2201100097, 2201100098, 2201100099, 2201100100,
       2201100104, 2201100105, 2201100118, 2201100139, 2201100140,
       2201100144, 2201100145, 2201100146, 2201100147, 2201100148,
       2201100149, 2201100150, 2201100154, 2201100155, 2201100156,
       2201100157, 2201100159, 2201100160, 2201100196, 2201100197,
       2201100198, 2201100199, 2201100200, 2201100204, 2201100205,
       2201100206, 2201100207, 2201100208, 2201100209, 2201100210,
       2201100214, 2201100215, 2201100216, 2201100217, 2201100218,
       2201100219, 2201100220, 2201100254, 2201100255, 2201100256,
       2201100257, 2201100258, 2201100259, 2201100260, 2201100

In [53]:
df_truth_11_2[['RunID', 'EventID']].nunique()

RunID       343
EventID    8000
dtype: int64

In [54]:
df_truth_11_2['MuonFilter_13'].value_counts()

MuonFilter_13
1    321550
0     81135
Name: count, dtype: int64

In [193]:
mucut_22010_1 = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied_filtered/Snowstorm/MuonLike/22010/"
mucut_22012_1 = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied_filtered/Snowstorm/MuonLike/22012/"

mucut_22010_1_t1 = mucut_22010_1 + "truth_1.parquet"
mucut_22010_1_1 = mucut_22010_1 + "1/"
mucut_22010_1_2 = mucut_22010_1 + "2/"

mucut_22012_1_t1 = mucut_22012_1 + "truth_1.parquet"
mucut_22012_1_1 = mucut_22012_1 + "1/"
mucut_22012_1_2 = mucut_22012_1 + "2/"

In [194]:
df_mucut_22012_1_t1 = convertParquetToDF(mucut_22012_1_t1)

In [195]:
df_mucut_22012_1_t1

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,azimuth_GNHighestEDaughter,dir_x_GNHighestEDaughter,dir_y_GNHighestEDaughter,dir_z_GNHighestEDaughter,pos_x_GNHighestEDaughter,pos_y_GNHighestEDaughter,pos_z_GNHighestEDaughter,time_GNHighestEDaughter,speed_GNHighestEDaughter,energy_GNHighestEDaughter
0,112000100000001,1,12,1,1,132,132,53755724,5,2,...,5,-1,1,0,923202,-1055929,-194442,-4714972,-1,2661784
1,112000100000002,2,12,1,1,313,445,25769004,3,1,...,3,1,-0,-1,-2100,184,1940,1415,0,15106847
2,112000100000003,3,12,1,1,279,724,6547825,5,0,...,5,-0,0,-1,-254,-498,-483,9765,0,6363586
3,112000100000004,4,12,1,1,741,1465,84461944,1,1,...,1,-0,-1,-1,1382,2362,1249,156,0,80826808
4,112000100000005,5,12,1,1,659,2124,27426738,6,0,...,6,-0,0,-1,-181,-666,1483,6329,0,20655186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26171,112000100029381,29381,12,1,15,264,486589,6344052,3,1,...,3,1,-0,-1,-498,-14,1303,6325,0,4508128
26172,112000100029382,29382,12,1,15,365,486954,4438848,1,1,...,1,-1,-0,-1,569,906,301,7033,0,3621763
26173,112000100029383,29383,12,1,15,232,487186,18671772,4,1,...,4,1,0,-1,416,-249,-682,12773,-1,657928
26174,112000100029384,29384,12,1,15,32,487218,12973503,2,3,...,2,0,-0,1,-2471008,3537996,-7384847,-28519658,-1,799604


In [196]:
df_mucut_22012_1_t1[1800:1820]

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,azimuth_GNHighestEDaughter,dir_x_GNHighestEDaughter,dir_y_GNHighestEDaughter,dir_z_GNHighestEDaughter,pos_x_GNHighestEDaughter,pos_y_GNHighestEDaughter,pos_z_GNHighestEDaughter,time_GNHighestEDaughter,speed_GNHighestEDaughter,energy_GNHighestEDaughter
1800,112000100001989,1989,12,1,1,53,683492,20478768,5,1,...,5,-1,1,-0,-458,255,-794,7959,0,19230556
1801,112000100001990,1990,12,1,1,303,683795,8063884,6,2,...,6,-1,0,1,5676659,-606885,-5188802,-25722242,-1,5356524
1802,112000100001991,1991,12,1,1,115,683910,5581592,5,1,...,5,0,1,-0,-332,-2870,319,1207,-1,3738963
1803,112000100001992,1992,12,1,1,13,683923,15163564,3,2,...,3,1,-0,0,-1488025,508488,-274725,-5315223,-1,6428593
1804,112000100001994,1994,12,1,1,320,684243,14815209,6,2,...,6,-1,0,1,5044612,-3795053,-7167856,-31848600,-1,333678
1805,112000100001995,1995,12,1,1,512,684755,8815127,5,1,...,5,-1,1,-0,2533,-2730,633,-985,0,8253728
1806,112000100001996,1996,12,1,1,956,685711,15973944,5,1,...,5,0,1,-1,153,453,-241,9540,-1,14688809
1807,112000100001997,1997,12,1,1,93,685804,16048499,5,1,...,5,-1,1,-0,3168,-5288,1140,-12046,0,14526796
1808,112000100001998,1998,12,1,1,884,686688,39477656,5,1,...,5,0,1,-1,-21,-1627,1920,2674,0,39021468
1809,112000100002001,2001,12,1,2,665,665,2212301,5,0,...,5,-0,0,-1,-74,-813,1902,4818,0,1210958


In [146]:
thorstencut_22010_1 = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied_filtered/Snowstorm/MuonLike_PureNu/22010/"
thorstencut_22012_1 = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied_filtered/Snowstorm/MuonLike_PureNu/22012/"

thorstencut_22010_t1 = thorstencut_22010_1 + "truth_1.parquet"
thorstencut_22010_1_1 = thorstencut_22010_1 + "1/"

thorstencut_22012_t1 = thorstencut_22012_1 + "truth_1.parquet"
thorstencut_22012_1_1 = thorstencut_22012_1 + "1/"

In [147]:
df_thorstencut_22010_t1 = convertParquetToDF(thorstencut_22010_t1)
df_thorstencut_22010_1_1 = convertParquetToDF(thorstencut_22010_1_1+"PMTfied_1.parquet")
df_thorstencut_22010_1_2 = convertParquetToDF(thorstencut_22010_1_1+"PMTfied_2.parquet")

df_thorstencut_22012_t1 = convertParquetToDF(thorstencut_22012_t1)
df_thorstencut_22012_1_1 = convertParquetToDF(thorstencut_22012_1_1+"PMTfied_1.parquet")
df_thorstencut_22012_1_2 = convertParquetToDF(thorstencut_22012_1_1+"PMTfied_2.parquet")
df_thorstencut_22012_1_3 = convertParquetToDF(thorstencut_22012_1_1+"PMTfied_3.parquet")

In [148]:
df_thorstencut_22010_t1[:20]

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,azimuth_GNHighestEDaughter,dir_x_GNHighestEDaughter,dir_y_GNHighestEDaughter,dir_z_GNHighestEDaughter,pos_x_GNHighestEDaughter,pos_y_GNHighestEDaughter,pos_z_GNHighestEDaughter,time_GNHighestEDaughter,speed_GNHighestEDaughter,energy_GNHighestEDaughter
0,110000100000006,6,10,1,1,111,111,5043,4,2,...,4,1,1,0,-1070,-1174,-1353,4146,0,4815
1,110000100000007,7,10,1,1,25,136,482,1,2,...,1,-1,-1,0,504,699,-326,8408,0,357
2,110000100000008,8,10,1,1,22,158,6637,1,2,...,1,-1,-1,-0,4871,5409,269,-13207,0,6620
3,110000100000011,11,10,1,1,30,188,7093,3,2,...,3,1,0,0,-3901,-706,-823,-2489,0,3749
4,110000100000012,12,10,1,1,130,318,444,1,1,...,1,-1,-1,-0,124,-19,-408,9972,0,261
5,110000100000013,13,10,1,1,251,569,4641,2,0,...,2,0,-0,-1,43,30,207,9795,-1,3976
6,110000100000017,17,10,1,1,35,604,3268,5,3,...,5,-0,1,1,255,-953,-1242,6753,0,2231
7,110000100000018,18,10,1,1,49,653,4674,1,2,...,1,-1,-1,-0,26,444,228,9644,0,4372
8,110000100000023,23,10,1,1,48,701,7485,0,2,...,6,-1,0,1,659,173,-1322,6017,0,6248
9,110000100000024,24,10,1,1,25,726,1482,5,2,...,5,-0,1,1,-336,265,-534,9622,-1,1119


In [149]:
df_thorstencut_22010_1_1

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,hlc4,hlc5,t1,t2,t3,t4,t5,T10,T50,sigmaT
0,110000100030158,30158,382,239,294,5,-151,19,0,1,...,-1,-1,10567,-1,-1,-1,-1,-1,-1,-1
1,110000100030158,30158,505,258,336,128,-132,61,0,1,...,-1,-1,11550,-1,-1,-1,-1,-1,-1,-1
2,110000100030158,30158,505,258,285,128,-132,10,0,1,...,-1,-1,10764,15851,-1,-1,-1,0,5087,2544
3,110000100030158,30158,505,258,268,128,-132,-8,0,1,...,-1,-1,10561,-1,-1,-1,-1,-1,-1,-1
4,110000100030158,30158,303,336,377,-74,-54,101,0,1,...,-1,-1,10780,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867698,110000100192784,192784,107,27,-497,-207,-115,-165,0,1,...,-1,-1,10429,-1,-1,-1,-1,-1,-1,-1
1867699,110000100192784,192784,113,-60,-273,-201,-202,59,0,1,...,-1,-1,12038,-1,-1,-1,-1,-1,-1,-1
1867700,110000100192784,192784,113,-60,-448,-201,-202,-116,0,1,...,-1,-1,11182,-1,-1,-1,-1,-1,-1,-1
1867701,110000100192784,192784,113,-60,-490,-201,-202,-158,0,1,...,-1,-1,11184,-1,-1,-1,-1,-1,-1,-1


In [150]:
df_thorstencut_22012_t1['MuonFilter_13'].nunique()

1

In [151]:
df_thorstencut_22012_t1[:10]

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,azimuth_GNHighestEDaughter,dir_x_GNHighestEDaughter,dir_y_GNHighestEDaughter,dir_z_GNHighestEDaughter,pos_x_GNHighestEDaughter,pos_y_GNHighestEDaughter,pos_z_GNHighestEDaughter,time_GNHighestEDaughter,speed_GNHighestEDaughter,energy_GNHighestEDaughter
0,112000100000001,1,12,1,1,132,132,53755724,5,2,...,5,-1,1,0,923202,-1055929,-194442,-4714972,-1,2661784
1,112000100000003,3,12,1,1,279,411,6547825,5,0,...,5,-0,0,-1,-254,-498,-483,9765,0,6363586
2,112000100000004,4,12,1,1,741,1152,84461944,1,1,...,1,-0,-1,-1,1382,2362,1249,156,0,80826808
3,112000100000005,5,12,1,1,659,1811,27426738,6,0,...,6,-0,0,-1,-181,-666,1483,6329,0,20655186
4,112000100000006,6,12,1,1,1071,2882,3670788,4,1,...,4,0,1,-1,302,-108,-220,9603,0,2756233
5,112000100000007,7,12,1,1,203,3085,6619170,0,1,...,0,-1,-0,-1,-479,314,-551,9349,0,6211112
6,112000100000009,9,12,1,1,16,3101,77955152,4,1,...,4,1,1,-0,366,-590,474,9007,-1,288167
7,112000100000010,10,12,1,1,252,3353,29289832,3,1,...,3,1,-0,-0,-4393,1678,1723,-5679,-1,21989224
8,112000100000011,11,12,1,1,90,3443,12322922,5,2,...,5,-0,1,-0,879,-1145,133,5318,0,9138703
9,112000100000013,13,12,1,1,204,3647,4634440,1,2,...,1,-1,-1,0,1070568,1712222,-332037,-6814788,-1,1039245


In [152]:
df_thorstencut_22012_t1.head

<bound method NDFrame.head of               event_no  original_event_no  subdirectory_no  part_no  shard_no  \
0      112000100000001                  1               12        1         1   
1      112000100000003                  3               12        1         1   
2      112000100000004                  4               12        1         1   
3      112000100000005                  5               12        1         1   
4      112000100000006                  6               12        1         1   
...                ...                ...              ...      ...       ...   
21183  112000100029380              29380               12        1        15   
21184  112000100029381              29381               12        1        15   
21185  112000100029383              29383               12        1        15   
21186  112000100029384              29384               12        1        15   
21187  112000100029385              29385               12        1        15  

In [153]:
df_thorstencut_22012_t1

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,azimuth_GNHighestEDaughter,dir_x_GNHighestEDaughter,dir_y_GNHighestEDaughter,dir_z_GNHighestEDaughter,pos_x_GNHighestEDaughter,pos_y_GNHighestEDaughter,pos_z_GNHighestEDaughter,time_GNHighestEDaughter,speed_GNHighestEDaughter,energy_GNHighestEDaughter
0,112000100000001,1,12,1,1,132,132,53755724,5,2,...,5,-1,1,0,923202,-1055929,-194442,-4714972,-1,2661784
1,112000100000003,3,12,1,1,279,411,6547825,5,0,...,5,-0,0,-1,-254,-498,-483,9765,0,6363586
2,112000100000004,4,12,1,1,741,1152,84461944,1,1,...,1,-0,-1,-1,1382,2362,1249,156,0,80826808
3,112000100000005,5,12,1,1,659,1811,27426738,6,0,...,6,-0,0,-1,-181,-666,1483,6329,0,20655186
4,112000100000006,6,12,1,1,1071,2882,3670788,4,1,...,4,0,1,-1,302,-108,-220,9603,0,2756233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21183,112000100029380,29380,12,1,15,762,8377099,4664040,0,2,...,0,-1,-0,1,4450148,1114529,-4306700,-20975290,-1,186903
21184,112000100029381,29381,12,1,15,264,8377363,6344052,3,1,...,3,1,-0,-1,-498,-14,1303,6325,0,4508128
21185,112000100029383,29383,12,1,15,232,8377595,18671772,4,1,...,4,1,0,-1,416,-249,-682,12773,-1,657928
21186,112000100029384,29384,12,1,15,32,8377627,12973503,2,3,...,2,0,-0,1,-2471008,3537996,-7384847,-28519658,-1,799604


In [ ]:
# calculate the cumsum along the row of the 'N_dom' column
df_thorstencut_22012_t1['N_dom_cumsum'] = df_thorstencut_22012_t1['N_doms'].cumsum()
df_thorstencut_22012_t1[['event_no','N_doms', 'offset', 'N_dom_cumsum']]

,event_no,N_doms,offset,N_dom_cumsum
0,112000100000001,132,132,132
1,112000100000003,279,411,411
2,112000100000004,741,1152,1152
3,112000100000005,659,1811,1811
4,112000100000006,1071,2882,2882
...,...,...,...,...
21183,112000100029380,762,8377099,8377099
21184,112000100029381,264,8377363,8377363
21185,112000100029383,232,8377595,8377595
21186,112000100029384,32,8377627,8377627


In [ ]:
df_thorstencut_22012_t1_1 = df_thorstencut_22012_t1[:1462]
df_thorstencut_22012_t1_1[['event_no','shard_no','N_doms', 'offset', 'N_dom_cumsum']]

,event_no,shard_no,N_doms,offset,N_dom_cumsum
0,112000100000001,1,132,132,132
1,112000100000003,1,279,411,411
2,112000100000004,1,741,1152,1152
3,112000100000005,1,659,1811,1811
4,112000100000006,1,1071,2882,2882
...,...,...,...,...,...
1457,112000100001994,1,320,557566,557566
1458,112000100001995,1,512,558078,558078
1459,112000100001996,1,956,559034,559034
1460,112000100001997,1,93,559127,559127


In [164]:
df_thorstencut_22012_t1_2 = df_thorstencut_22012_t1[1463:2924]
df_thorstencut_22012_t1_2[['event_no','shard_no','N_doms', 'offset', 'N_dom_cumsum']]

,event_no,shard_no,N_doms,offset,N_dom_cumsum
1463,112000100002003,2,231,560479,560479
1464,112000100002005,2,69,560548,560548
1465,112000100002007,2,74,560622,560622
1466,112000100002008,2,57,560679,560679
1467,112000100002010,2,87,560766,560766
...,...,...,...,...,...
2919,112000100003993,2,501,1127806,1127806
2920,112000100003994,2,932,1128738,1128738
2921,112000100003996,2,1113,1129851,1129851
2922,112000100003997,2,793,1130644,1130644


In [170]:
count_thorstencut_22012_1_1 = df_thorstencut_22012_1_1['event_no'].value_counts().sort_index()
count_thorstencut_22012_1_1

event_no
112000100000001     132
112000100000003     279
112000100000004     741
112000100000005     659
112000100000006    1071
                   ... 
112000100001994     320
112000100001995     512
112000100001996     956
112000100001997      93
112000100001998     884
Name: count, Length: 1462, dtype: int64

In [ ]:
count_thorstencut_22012_1_1

In [173]:
count_thorstencut_22012_1_2 = df_thorstencut_22012_1_2['event_no'].value_counts().sort_index()
count_thorstencut_22012_1_2

event_no
112000100002002     237
112000100002003     231
112000100002005      69
112000100002007      74
112000100002008      57
                   ... 
112000100003993     501
112000100003994     932
112000100003996    1113
112000100003997     793
112000100003998     665
Name: count, Length: 1462, dtype: int64

In [171]:
df_thorstencut_22012_t1

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,dir_x_GNHighestEDaughter,dir_y_GNHighestEDaughter,dir_z_GNHighestEDaughter,pos_x_GNHighestEDaughter,pos_y_GNHighestEDaughter,pos_z_GNHighestEDaughter,time_GNHighestEDaughter,speed_GNHighestEDaughter,energy_GNHighestEDaughter,N_dom_cumsum
0,112000100000001,1,12,1,1,132,132,53755724,5,2,...,-1,1,0,923202,-1055929,-194442,-4714972,-1,2661784,132
1,112000100000003,3,12,1,1,279,411,6547825,5,0,...,-0,0,-1,-254,-498,-483,9765,0,6363586,411
2,112000100000004,4,12,1,1,741,1152,84461944,1,1,...,-0,-1,-1,1382,2362,1249,156,0,80826808,1152
3,112000100000005,5,12,1,1,659,1811,27426738,6,0,...,-0,0,-1,-181,-666,1483,6329,0,20655186,1811
4,112000100000006,6,12,1,1,1071,2882,3670788,4,1,...,0,1,-1,302,-108,-220,9603,0,2756233,2882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21183,112000100029380,29380,12,1,15,762,8377099,4664040,0,2,...,-1,-0,1,4450148,1114529,-4306700,-20975290,-1,186903,8377099
21184,112000100029381,29381,12,1,15,264,8377363,6344052,3,1,...,1,-0,-1,-498,-14,1303,6325,0,4508128,8377363
21185,112000100029383,29383,12,1,15,232,8377595,18671772,4,1,...,1,0,-1,416,-249,-682,12773,-1,657928,8377595
21186,112000100029384,29384,12,1,15,32,8377627,12973503,2,3,...,0,-0,1,-2471008,3537996,-7384847,-28519658,-1,799604,8377627


In [90]:
df_thorstencut_22012_1_1[:-1]

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,hlc4,hlc5,t1,t2,t3,t4,t5,T10,T50,sigmaT
0,112000100000001,1,-335,-424,194,148,-193,116,0,1,...,1,-1,11070,11147,11825,12319,-1,0,755,513
1,112000100000001,1,-335,-424,177,148,-193,99,0,1,...,-1,-1,10888,10974,12880,-1,-1,0,86,919
2,112000100000001,1,-335,-424,160,148,-193,82,0,1,...,1,1,11348,11446,11506,11559,12655,0,158,481
3,112000100000001,1,-335,-424,143,148,-193,65,0,1,...,-1,-1,12248,12935,-1,-1,-1,0,0,344
4,112000100000001,1,-335,-424,126,148,-193,48,0,1,...,-1,-1,11197,11385,12005,-1,-1,0,188,345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560005,112000100001998,1998,-11,7,150,-84,-194,-106,0,1,...,-1,-1,12898,19857,-1,-1,-1,0,0,3480
560006,112000100001998,1998,-11,7,140,-84,-194,-116,0,1,...,1,1,12329,12365,12756,12893,13050,0,564,397
560007,112000100001998,1998,-11,7,130,-84,-194,-126,0,1,...,-1,-1,12963,-1,-1,-1,-1,-1,-1,-1
560008,112000100001998,1998,-11,7,120,-84,-194,-136,0,1,...,-1,-1,12806,12828,13473,-1,-1,0,22,309


In [101]:
df_thorstencut_22012_1_1

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,hlc4,hlc5,t1,t2,t3,t4,t5,T10,T50,sigmaT
0,112000100000001,1,-335,-424,194,148,-193,116,0,1,...,1,-1,11070,11147,11825,12319,-1,0,755,513
1,112000100000001,1,-335,-424,177,148,-193,99,0,1,...,-1,-1,10888,10974,12880,-1,-1,0,86,919
2,112000100000001,1,-335,-424,160,148,-193,82,0,1,...,1,1,11348,11446,11506,11559,12655,0,158,481
3,112000100000001,1,-335,-424,143,148,-193,65,0,1,...,-1,-1,12248,12935,-1,-1,-1,0,0,344
4,112000100000001,1,-335,-424,126,148,-193,48,0,1,...,-1,-1,11197,11385,12005,-1,-1,0,188,345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560006,112000100001998,1998,-11,7,140,-84,-194,-116,0,1,...,1,1,12329,12365,12756,12893,13050,0,564,397
560007,112000100001998,1998,-11,7,130,-84,-194,-126,0,1,...,-1,-1,12963,-1,-1,-1,-1,-1,-1,-1
560008,112000100001998,1998,-11,7,120,-84,-194,-136,0,1,...,-1,-1,12806,12828,13473,-1,-1,0,22,309
560009,112000100001998,1998,-11,7,110,-84,-194,-146,0,1,...,-1,-1,13180,13518,13528,-1,-1,0,338,162


In [107]:
df_thorstencut_22012_1_2

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,hlc4,hlc5,t1,t2,t3,t4,t5,T10,T50,sigmaT
0,112000100002002,2002,-256,-521,-202,52,-99,261,0,1,...,-1,-1,12076,-1,-1,-1,-1,-1,-1,-1
1,112000100002002,2002,-256,-521,-219,52,-99,244,0,1,...,-1,-1,11467,-1,-1,-1,-1,-1,-1,-1
2,112000100002002,2002,-256,-521,-253,52,-99,210,0,1,...,-1,-1,11996,-1,-1,-1,-1,-1,-1,-1
3,112000100002002,2002,-256,-521,-287,52,-99,176,0,1,...,1,1,11242,11390,11833,12451,14151,0,591,2660
4,112000100002002,2002,-256,-521,-304,52,-99,159,0,1,...,-1,-1,12906,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571293,112000100003998,3998,57,-106,-372,144,360,-555,0,1,...,-1,-1,17638,-1,-1,-1,-1,-1,-1,-1
571294,112000100003998,3998,57,-106,-400,144,360,-583,0,1,...,-1,-1,20096,-1,-1,-1,-1,-1,-1,-1
571295,112000100003998,3998,57,-106,-463,144,360,-646,0,1,...,-1,-1,20326,-1,-1,-1,-1,-1,-1,-1
571296,112000100003998,3998,-10,-80,-472,77,386,-656,0,1,...,-1,-1,19268,-1,-1,-1,-1,-1,-1,-1


In [106]:
df_thorstencut_22012_1_2[df_thorstencut_22012_1_2['event_no'] == 112000100002047]

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,hlc4,hlc5,t1,t2,t3,t4,t5,T10,T50,sigmaT
10720,112000100002047,2047,-256,-521,-236,-108,-427,165,0,1,...,-1,-1,14342,-1,-1,-1,-1,-1,-1,-1
10721,112000100002047,2047,-256,-521,-287,-108,-427,114,0,1,...,-1,-1,13912,-1,-1,-1,-1,-1,-1,-1
10722,112000100002047,2047,-256,-521,-338,-108,-427,62,0,1,...,-1,-1,13972,15739,-1,-1,-1,0,0,884
10723,112000100002047,2047,-256,-521,-372,-108,-427,28,0,1,...,-1,-1,13746,-1,-1,-1,-1,-1,-1,-1
10724,112000100002047,2047,-256,-521,-457,-108,-427,-57,0,1,...,-1,-1,14070,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12049,112000100002047,2047,-11,7,-472,137,100,-72,0,1,...,1,1,12241,12257,12267,12272,12282,115,575,2353
12050,112000100002047,2047,-11,7,-479,137,100,-79,0,1,...,1,1,12210,12231,12236,12250,12266,123,549,1981
12051,112000100002047,2047,-11,7,-486,137,100,-86,0,1,...,1,1,12229,12241,12248,12256,12259,106,535,2242
12052,112000100002047,2047,-11,7,-493,137,100,-93,0,1,...,1,1,12230,12239,12258,12263,12270,103,477,2534


In [103]:
df_thorstencut_22012_1_3

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,hlc4,hlc5,t1,t2,t3,t4,t5,T10,T50,sigmaT
0,112000100004000,4000,-256,-521,-321,181,-348,150,0,1,...,-1,-1,16759,-1,-1,-1,-1,-1,-1,-1
1,112000100004000,4000,-256,-521,-372,181,-348,99,0,1,...,-1,-1,14223,-1,-1,-1,-1,-1,-1,-1
2,112000100004000,4000,-256,-521,-406,181,-348,65,0,1,...,-1,-1,14462,-1,-1,-1,-1,-1,-1,-1
3,112000100004000,4000,-256,-521,-423,181,-348,48,0,1,...,-1,-1,14709,-1,-1,-1,-1,-1,-1,-1
4,112000100004000,4000,-256,-521,-457,181,-348,14,0,1,...,-1,-1,13481,16619,17431,-1,-1,0,3138,1703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597052,112000100005999,5999,-11,7,-416,-466,-52,-127,0,1,...,-1,-1,12867,13142,13402,-1,-1,0,275,218
597053,112000100005999,5999,-11,7,-430,-466,-52,-141,0,1,...,-1,-1,14830,15382,-1,-1,-1,0,0,276
597054,112000100005999,5999,-11,7,-444,-466,-52,-155,0,1,...,-1,-1,14981,-1,-1,-1,-1,-1,-1,-1
597055,112000100005999,5999,-11,7,-472,-466,-52,-183,0,1,...,-1,-1,13488,-1,-1,-1,-1,-1,-1,-1
